## Arrumando os dados

Realizei a organização dos dados nessa primeira etapa.

In [1]:
from mwrogue.esports_client import EsportsClient
from mwrogue.auth_credentials import AuthCredentials

credentials = AuthCredentials(user_file="me2")
site = EsportsClient('lol', credentials=credentials)

In [2]:
# Pegando os dados da Leagupedia
response = site.cargo_client.query(
    tables = "ScoreboardGames=SG",
    fields="SG.Tournament, " \
            "SG.Team1, " \
            "SG.Team2, " \
            "SG.WinTeam, " \
            "SG.LossTeam, " \
            "SG.Team1Picks, " \
            "SG.Team2Picks, " \
            "SG.Team1Players, " \
            "SG.Team2Players, " \
            "SG.Team1Dragons, " \
            "SG.Team2Dragons, " \
            "SG.Team1Clouds, " \
            "SG.Team1Infernals, " \
            "SG.Team1Mountains, " \
            "SG.Team1Oceans, " \
            "SG.Team1Hextechs, " \
            "SG.Team1Chemtechs, " \
            "SG.Team1Elders, " \
            "SG.Team2Clouds, " \
            "SG.Team2Infernals, " \
            "SG.Team2Mountains," \
            "SG.Team2Oceans," \
            "SG.Team2Hextechs," \
            "SG.Team2Chemtechs," \
            "SG.Team2Elders," \
            "SG.Team1Barons," \
            "SG.Team2Barons," \
            "SG.Team1Towers," \
            "SG.Team2Towers," \
            "SG.Team1Gold," \
            "SG.Team2Gold," \
            "SG.Team1Kills," \
            "SG.Team2Kills," \
            "SG.Team1RiftHeralds," \
            "SG.Team2RiftHeralds," \
            "SG.Team1VoidGrubs," \
            "SG.Team2VoidGrubs," \
            "SG.Team1Atakhans," \
            "SG.Team2Atakhans",
    where='SG.Tournament = "IDL Kings Lendas Season 3"'
)

# Validando as informações
for rows in response:
    for item, value in rows.items():
        print(item, value)

Tournament IDL Kings Lendas Season 3
Team1 100Vices
Team2 ÉanDG
WinTeam ÉanDG
LossTeam 100Vices
Team1Picks Yorick,Skarner,Viktor,Zeri,Neeko
Team2Picks K'Sante,Pantheon,Hwei,Sivir,Karma
Team1Players Pijack,sarolu,Takeshi,Celo,Cavalo
Team2Players zynts,randal,Qats,brTT,Telas
Team1Dragons 2
Team2Dragons 4
Team1Clouds 0
Team1Infernals 0
Team1Mountains 1
Team1Oceans 0
Team1Hextechs 1
Team1Chemtechs 0
Team1Elders 0
Team2Clouds 0
Team2Infernals 0
Team2Mountains 3
Team2Oceans 0
Team2Hextechs 0
Team2Chemtechs 1
Team2Elders 0
Team1Barons 0
Team2Barons 2
Team1Towers 5
Team2Towers 10
Team1Gold 69069
Team2Gold 76822
Team1Kills 15
Team2Kills 38
Team1RiftHeralds 0
Team2RiftHeralds 1
Team1VoidGrubs 3
Team2VoidGrubs 0
Team1Atakhans 0
Team2Atakhans 1
Tournament IDL Kings Lendas Season 3
Team1 100Vices
Team2 FONatic
WinTeam FONatic
LossTeam 100Vices
Team1Picks Sion,Jarvan IV,Ryze,Xayah,Rakan
Team2Picks Ambessa,Wukong,Taliyah,Zeri,Alistar
Team1Players Pijack,sarolu,Takeshi,Celo,Cavalo
Team2Players zekas,D

In [3]:
# Importando o pandas e mantendo os dados brutos em um DataFrame para não ficar importando os dados via API o tempo inteiro
# Há um limite para uso da API

import pandas as pd

dataframe = pd.DataFrame(response)

static_dataframe = dataframe

In [4]:
# Criando o nome para as novas colunas do DataFrame
# Elas serão geradas da separação das colunas de Picks e Players

cols_1 = ["Team1_top","Team1_jg","Team1_mid","Team1_adc","Team1_sup"]
cols_2 = ["Team2_top","Team2_jg","Team2_mid","Team2_adc","Team2_sup"]
cols_3 = ["Team1player_top","Team1player_jg", "Team1player_mid","Team1player_adc","Team1player_sup"]
cols_4 = ["Team2player_top","Team2player_jg", "Team2player_mid","Team2player_adc","Team2player_sup"]

In [5]:
# Separando os dados de pick e players dos 2 times para análise detalhada por players e campeões
dataframe[cols_1] = dataframe["Team1Picks"].str.split(",", expand=True, n=4)
dataframe[cols_2] = dataframe["Team2Picks"].str.split(",", expand=True, n=4)
dataframe[cols_3] = dataframe["Team1Players"].str.split(",", expand=True, n=4)
dataframe[cols_4] = dataframe["Team2Players"].str.split(",", expand=True, n=4)

# Droppando as colunas que foram separadas (Não são mais úteis)
dataframe = dataframe.drop(columns=["Team1Picks","Team2Picks","Team1Players","Team2Players"])

In [6]:
# Aplicando uma função lambda de ordenação nas colunas referentes à posição jg no dataframe
# É necessário ordenar para poder realizar a contagem de quantas vezes determinada MatchUp aconteceu
jg_matchup = dataframe[["Team1_jg","Team2_jg"]].apply(lambda s: tuple(sorted(s)), axis=1)

# Contando a quantidade de vezes que uma determinada matchup aconteceu
jg_matchup.value_counts()

(Wukong, Xin Zhao)       8
(Jarvan IV, Wukong)      7
(Pantheon, Xin Zhao)     7
(Sejuani, Xin Zhao)      5
(Jarvan IV, Xin Zhao)    5
(Pantheon, Skarner)      5
(Skarner, Wukong)        4
(Qiyana, Trundle)        4
(Nocturne, Trundle)      4
(Jarvan IV, Pantheon)    4
(Jarvan IV, Trundle)     4
(Sejuani, Vi)            4
(Trundle, Wukong)        4
(Jax, Xin Zhao)          3
(Nocturne, Xin Zhao)     3
(Jarvan IV, Nocturne)    3
(Sejuani, Wukong)        3
(Skarner, Vi)            2
(Trundle, Xin Zhao)      2
(Maokai, Xin Zhao)       2
(Vi, Xin Zhao)           2
(Trundle, Vi)            2
(Vi, Wukong)             2
(Nocturne, Pantheon)     2
(Pantheon, Trundle)      2
(Jarvan IV, Skarner)     2
(Jarvan IV, Sejuani)     2
(Poppy, Skarner)         1
(Dr. Mundo, Skarner)     1
(Nocturne, Qiyana)       1
(Pantheon, Vi)           1
(Jarvan IV, Qiyana)      1
(Nocturne, Sejuani)      1
(Jax, Nocturne)          1
(Naafiri, Trundle)       1
(Lillia, Zyra)           1
(Nocturne, Skarner)      1
(

In [7]:
df_jg_matchup = pd.DataFrame(jg_matchup.value_counts()).reset_index()

df_jg_matchup = df_jg_matchup.rename(columns={"index":"Matchup"})

In [8]:
df_jg_matchup

,Matchup,count
0,"(Wukong, Xin Zhao)",8
1,"(Jarvan IV, Wukong)",7
2,"(Pantheon, Xin Zhao)",7
3,"(Sejuani, Xin Zhao)",5
4,"(Jarvan IV, Xin Zhao)",5
5,"(Pantheon, Skarner)",5
6,"(Skarner, Wukong)",4
7,"(Qiyana, Trundle)",4
8,"(Nocturne, Trundle)",4
9,"(Jarvan IV, Pantheon)",4


In [9]:
print("Quantidade de confrontos únicos: ", df_jg_matchup.shape[0])
print("Porcentagem dos 5 primeiros confrontos em relação ao total: ", (df_jg_matchup["count"].head(5).sum()/df_jg_matchup["count"].sum()).round(3)*100,"%")

Quantidade de confrontos únicos:  54
Porcentagem dos 5 primeiros confrontos em relação ao total:  25.8 %


In [28]:
# Analisando os dados relativos às MatchUps

# Alocando o nome dos campeões
champ1, champ2 = "Jarvan IV", "Xin Zhao"

# Preparando a análise comparativa dentro das máscaras
df_jg_mask = ((dataframe["Team1_jg"].eq(champ1) & dataframe["Team2_jg"].eq(champ2)) | (dataframe["Team1_jg"].eq(champ2) & dataframe["Team2_jg"].eq(champ1)))

# Aplicando a máscara dentro do Dataframe e criando um dataframe específico apenas para essa análise
most_jg_matchup_dataframe = dataframe[df_jg_mask]

In [29]:
# Realizando um slice de comparativo dos matchups da jungle
most_jg_matchup_dataframe[["Team1","Team2","WinTeam","Team1_jg", "Team2_jg", "Team1player_jg", "Team2player_jg","Team1Dragons","Team2Dragons","Team1Barons","Team2Barons","Team1RiftHeralds","Team2RiftHeralds","Team1VoidGrubs","Team2VoidGrubs","Team1Atakhans","Team2Atakhans"]]

,Team1,Team2,WinTeam,Team1_jg,Team2_jg,Team1player_jg,Team2player_jg,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1RiftHeralds,Team2RiftHeralds,Team1VoidGrubs,Team2VoidGrubs,Team1Atakhans,Team2Atakhans
14,ÉanDG,Oreiudos Esports,ÉanDG,Jarvan IV,Xin Zhao,randal,Anato,3,0,1,0,1,0,3,0,1,0
56,Gen GG,Oreiudos Esports,Gen GG,Xin Zhao,Jarvan IV,SamKz,Aegis (Gabriel Lemos),2,0,0,0,1,0,3,0,1,0
111,Tepei Esports,Vôs Grandes,Vôs Grandes,Xin Zhao,Jarvan IV,Dizin,stiner,1,3,0,1,0,1,3,0,0,1
115,Vôs Grandes,FONatic,FONatic,Jarvan IV,Xin Zhao,stiner,Drakehero,0,5,0,1,0,1,1,2,0,1
119,Vôs Grandes,Karmine Cospe,Karmine Cospe,Jarvan IV,Xin Zhao,stiner,Shini,0,4,0,1,0,1,1,2,0,1


In [12]:
dataframe[["Team1_jg","Team2_jg"]].stack().value_counts()

Xin Zhao     39
Wukong       32
Jarvan IV    29
Trundle      26
Pantheon     24
Sejuani      17
Skarner      17
Vi           17
Nocturne     17
Qiyana        7
Jax           5
Graves        4
Maokai        3
Naafiri       2
Poppy         2
Zyra          2
Dr. Mundo     1
Lillia        1
Lee Sin       1
Ivern         1
Zed           1
Name: count, dtype: int64

#### A partir desse ponto os dados visuais foram tratados no Canva para auxiliar na visualização das informações relativas à matchup na Jungle

----

## O famoso Atakhan da Kings Lendas

Bem, apesar da brincadeira relativo ao objetivo dentro do jogo, essa análise busca entender quem é o jogador mais efetivo realizando esse objetivo na Jungle, com qual campeão ele é mais efetivo e qual o panorama geral de vitória ao garantir o efeito positivo.

In [13]:
# Criando as máscaras para analisar os dados de times vencedores
mask1 = dataframe["WinTeam"].eq(dataframe["Team1"])
mask2 = dataframe["WinTeam"].eq(dataframe["Team2"])

In [14]:
# Criando um novo DataFrame apenas para análise exclusiva do Atakhan
# .copy() utilizado para evitar problema de indexação dentro de uma fatia do dataframe

atakhan_dataframe = dataframe[["WinTeam", "Team1", "Team2", "Team1_jg", "Team2_jg", "Team1player_jg", "Team2player_jg","Team1Atakhans", "Team2Atakhans"]].copy()

In [15]:
import numpy as np

In [16]:
# Criando as colunas com os dados dos picks, players e quantidade de atakhans dos vencedores
atakhan_dataframe["WinPlayer_jg"] = np.where(mask1, atakhan_dataframe["Team1player_jg"], np.where(mask2, atakhan_dataframe["Team2player_jg"], np.nan))

atakhan_dataframe["WinPick_jg"] = np.where(mask1, atakhan_dataframe["Team1_jg"], np.where(mask2, atakhan_dataframe["Team2_jg"], np.nan))

atakhan_dataframe["WinTeamAtakhans"] = np.where(mask1, atakhan_dataframe["Team1Atakhans"], np.where(mask2, atakhan_dataframe["Team2Atakhans"], np.nan))

In [17]:
# Analisando o resultado final do DataFrame
atakhan_dataframe

,WinTeam,Team1,Team2,Team1_jg,Team2_jg,Team1player_jg,Team2player_jg,Team1Atakhans,Team2Atakhans,WinPlayer_jg,WinPick_jg,WinTeamAtakhans
0,ÉanDG,100Vices,ÉanDG,Skarner,Pantheon,sarolu,randal,0,1,randal,Pantheon,1
1,FONatic,100Vices,FONatic,Jarvan IV,Wukong,sarolu,Drakehero,0,1,Drakehero,Wukong,1
2,G12,100Vices,G12,Qiyana,Nocturne,sarolu,Minerva,0,1,Minerva,Nocturne,1
3,100Vices,100Vices,Gen GG,Skarner,Dr. Mundo,sarolu,SamKz,1,0,sarolu,Skarner,1
4,100Vices,100Vices,Karmine Cospe,Xin Zhao,Vi,sarolu,Shini,0,1,sarolu,Xin Zhao,0
...,...,...,...,...,...,...,...,...,...,...,...,...
119,Karmine Cospe,Vôs Grandes,Karmine Cospe,Jarvan IV,Xin Zhao,stiner,Shini,0,1,Shini,Xin Zhao,1
120,Karmine Cospe,Vôs Grandes,Karmine Cospe,Vi,Poppy,stiner,Shini,0,1,Shini,Poppy,1
121,Oreiudos Esports,Vôs Grandes,Oreiudos Esports,Xin Zhao,Pantheon,stiner,Anato,0,1,Anato,Pantheon,1
122,Vôs Grandes,Vôs Grandes,Oreiudos Esports,Trundle,Zyra,stiner,Anato,1,0,stiner,Trundle,1


In [18]:
# Criando um DataFrame simples apenas para deixar o resultado estilizado
a = pd.DataFrame(atakhan_dataframe[["WinTeam","WinPlayer_jg","WinPick_jg","WinTeamAtakhans"]].value_counts())

a.head(15)

count
WinTeam          WinPlayer_jg          WinPick_jg WinTeamAtakhans       
Gen GG           SamKz                 Xin Zhao   1                    5
G12              Minerva               Trundle    1                    4
                                       Xin Zhao   1                    4
Karmine Cospe    Shini                 Trundle    1                    4
Vôs Grandes      stiner                Pantheon   1                    4
100Vices         sarolu                Jarvan IV  1                    3
Karmine Cospe    Shini                 Wukong     1                    3
Gen GG           SamKz                 Skarner    1                    3
FONatic          Drakehero             Xin Zhao   1                    3
Tepei Esports    Dizin                 Trundle    1                    3
ÉanDG            randal                Wukong     1                    3
Gen GG           SamKz                 Wukong     1                    2
Oreiudos Esports Aegis (Gabriel Lemos) Graves     1                    2
FONatic          Drakehero             Trundle    0                    2
Vôs Grandes      stiner                Jarvan IV  1                    2

In [19]:
# Analisando a quantidade de vezes que um determinado campeão ganhou e garantiu o Atakhan na partida
atakhan_dataframe.groupby("WinPick_jg")["WinTeamAtakhans"].value_counts().sort_values(ascending=False)

WinPick_jg  WinTeamAtakhans
Xin Zhao    1                  19
Trundle     1                  16
Wukong      1                  12
Jarvan IV   1                  12
Pantheon    1                   8
Skarner     1                   8
Trundle     0                   6
Vi          1                   6
Nocturne    1                   5
Pantheon    0                   5
Xin Zhao    0                   4
Wukong      0                   4
Poppy       1                   2
Jarvan IV   0                   2
Graves      1                   2
Jax         1                   2
Nocturne    0                   2
Sejuani     0                   2
Lee Sin     1                   1
Maokai      1                   1
Qiyana      0                   1
Sejuani     1                   1
Qiyana      1                   1
Vi          0                   1
Zyra        1                   1
Name: count, dtype: int64

In [20]:
# Analisando quantas vezes um determinado campeão da jungle foi escolhido
# Essa análise já foi feita anteriormente, trouxe novamente pois pretendo utilizar ela no visual
dataframe[["Team1_jg","Team2_jg"]].stack().value_counts()

Xin Zhao     39
Wukong       32
Jarvan IV    29
Trundle      26
Pantheon     24
Sejuani      17
Skarner      17
Vi           17
Nocturne     17
Qiyana        7
Jax           5
Graves        4
Maokai        3
Naafiri       2
Poppy         2
Zyra          2
Dr. Mundo     1
Lillia        1
Lee Sin       1
Ivern         1
Zed           1
Name: count, dtype: int64

In [21]:
atakhan_dataframe.groupby("WinPlayer_jg")["WinTeamAtakhans"].value_counts().sort_values(ascending=False)

WinPlayer_jg           WinTeamAtakhans
Shini                  1                  14
SamKz                  1                  13
Minerva                1                  13
stiner                 1                  12
randal                 1                  11
Accez                  1                   8
Drakehero              1                   8
                       0                   7
sarolu                 1                   6
SamKz                  0                   6
Dizin                  1                   6
Aegis (Gabriel Lemos)  1                   4
stiner                 0                   3
randal                 0                   3
Shini                  0                   3
Accez                  0                   2
Anato                  1                   2
sarolu                 0                   2
Aegis (Gabriel Lemos)  0                   1
Name: count, dtype: int64